In [78]:
import pandas as pd
import cobra
from cobra.io import read_sbml_model
from corda import CORDA
from corda import reaction_confidence
from cobra import Model, Reaction, Metabolite
from gprofiler import GProfiler
import numpy as np
import os

In [4]:
input_model = "/home/users/lzehetner/data/human1/human1.xml"
mod = cobra.io.read_sbml_model(input_model)

In [70]:
input_transcriptome = "/home/users/lzehetner/data/paper4_aav/HEK_Dietmair/GSE36094_processed_data_filtered_by_detection-pvalue.txt"
transcriptome = pd.read_csv(input_transcriptome, sep = "\t")

In [71]:
transcriptome

,ID_REF,Sample 1,Sample 2,Sample 3,Sample 4,Sample 5,Sample 6,Sample 7,Sample 8,Sample 9,...,Sample 15,Sample 16,Sample 17,Sample 18,Sample 19,Sample 20,Sample 21,Sample 22,Sample 23,Sample 24
0,ILMN_1796063,10.909568,10.686033,10.484470,10.580420,10.821422,10.667875,10.668543,10.657857,10.719120,...,10.596481,10.520208,10.625893,10.597385,10.538295,10.604912,10.533972,10.763537,10.773874,10.608606
1,ILMN_1811966,7.509697,7.657644,7.697281,7.639011,7.520340,7.779817,7.551836,7.622506,7.843433,...,7.583145,7.832166,7.449888,7.727502,7.499623,7.722397,7.450478,7.417768,7.532130,7.273337
2,ILMN_1793729,9.661065,9.461523,9.441575,9.344593,9.432135,9.592894,9.394488,9.453710,9.514681,...,9.747644,9.671438,9.725853,9.601156,9.608487,9.331044,9.712728,9.328784,9.612157,9.367705
3,ILMN_1682799,10.749738,10.804832,10.786887,10.788665,10.758936,10.857105,10.866369,10.888417,10.710554,...,10.896396,10.775869,10.874435,10.843286,10.763537,10.696173,10.606106,10.835450,10.738942,10.712946
4,ILMN_3305563,7.416676,7.321956,7.551842,7.406942,7.540410,7.559454,7.274985,7.393043,7.657635,...,7.360854,7.349248,7.507573,7.306957,7.498274,7.497078,7.517227,7.358917,7.665639,7.395817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17831,ILMN_1791388,10.670905,10.730608,10.690599,10.537213,10.745895,10.628369,10.572055,10.545107,10.806923,...,10.670905,10.727159,10.720283,10.646798,10.553275,10.578889,10.429728,10.546045,10.398527,10.448278
17832,ILMN_1754126,8.525712,8.671791,8.767685,8.525182,8.754860,8.565907,8.493229,8.818182,8.670027,...,8.653864,8.502985,8.628427,8.794829,8.486958,8.668961,8.577495,8.505847,8.700040,8.628084
17833,ILMN_1707464,8.666101,8.916163,9.091163,9.222082,9.132319,9.230268,9.061613,9.001067,9.265903,...,9.209716,8.861835,9.216765,8.960703,9.072636,9.199874,9.227783,8.994122,9.044997,9.123413
17834,ILMN_1785618,8.368973,8.307045,8.356303,8.447054,8.453019,8.505772,8.381081,8.329230,8.139021,...,8.391881,8.492079,8.271358,8.269878,8.375524,8.255512,8.340678,8.181179,8.080509,8.288850


In [72]:
Ilmn_ID = transcriptome['ID_REF'].tolist()

In [73]:
gp = GProfiler(return_dataframe=True)
converted = gp.convert(organism='hsapiens',
            query=Ilmn_ID,
            target_namespace='ENSG')

convert_dict = converted.set_index('incoming').to_dict()['converted']

replaced = transcriptome.replace({"ID_REF": convert_dict})

# df = df[df["team"].str.contains("Team 1") == False]
tr = replaced[replaced["ID_REF"].str.contains("None")==False]

In [74]:
tr

,ID_REF,Sample 1,Sample 2,Sample 3,Sample 4,Sample 5,Sample 6,Sample 7,Sample 8,Sample 9,...,Sample 15,Sample 16,Sample 17,Sample 18,Sample 19,Sample 20,Sample 21,Sample 22,Sample 23,Sample 24
0,ENSG00000166326,10.909568,10.686033,10.484470,10.580420,10.821422,10.667875,10.668543,10.657857,10.719120,...,10.596481,10.520208,10.625893,10.597385,10.538295,10.604912,10.533972,10.763537,10.773874,10.608606
1,ENSG00000214584,7.509697,7.657644,7.697281,7.639011,7.520340,7.779817,7.551836,7.622506,7.843433,...,7.583145,7.832166,7.449888,7.727502,7.499623,7.722397,7.450478,7.417768,7.532130,7.273337
2,ENSG00000167173,9.661065,9.461523,9.441575,9.344593,9.432135,9.592894,9.394488,9.453710,9.514681,...,9.747644,9.671438,9.725853,9.601156,9.608487,9.331044,9.712728,9.328784,9.612157,9.367705
3,ENSG00000138134,10.749738,10.804832,10.786887,10.788665,10.758936,10.857105,10.866369,10.888417,10.710554,...,10.896396,10.775869,10.874435,10.843286,10.763537,10.696173,10.606106,10.835450,10.738942,10.712946
5,ENSG00000187867,7.551702,7.527827,7.771663,7.436065,7.660663,7.658081,7.817061,7.926140,7.738751,...,7.456589,7.224724,7.546217,7.834886,7.414538,7.664187,7.760509,7.655134,8.109273,7.644048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17831,ENSG00000142409,10.670905,10.730608,10.690599,10.537213,10.745895,10.628369,10.572055,10.545107,10.806923,...,10.670905,10.727159,10.720283,10.646798,10.553275,10.578889,10.429728,10.546045,10.398527,10.448278
17832,ENSG00000189410,8.525712,8.671791,8.767685,8.525182,8.754860,8.565907,8.493229,8.818182,8.670027,...,8.653864,8.502985,8.628427,8.794829,8.486958,8.668961,8.577495,8.505847,8.700040,8.628084
17833,ENSG00000290851,8.666101,8.916163,9.091163,9.222082,9.132319,9.230268,9.061613,9.001067,9.265903,...,9.209716,8.861835,9.216765,8.960703,9.072636,9.199874,9.227783,8.994122,9.044997,9.123413
17834,ENSG00000183963,8.368973,8.307045,8.356303,8.447054,8.453019,8.505772,8.381081,8.329230,8.139021,...,8.391881,8.492079,8.271358,8.269878,8.375524,8.255512,8.340678,8.181179,8.080509,8.288850


In [75]:
tr = tr.set_index(tr.columns[0])

In [80]:
np_ind = [6,7,8,9,10,11,18,19,20,21,22,23]
# Use list comprehension to select the columns by these indices
np_col = [tr.columns[i] for i in np_ind]
tr_np = tr[np_col]
col_names = ['NP_TP2_1', 'NP_TP2_2', 'NP_TP2_3', 'NP_TP3_1', 'NP_TP3_2', 'NP_TP3_3', 'NP_TP1_1', 'NP_TP1_2', 'NP_TP1_3', 'NP_TP4_1', 'NP_TP4_2', 'NP_TP4_3']  # Add your actual new names here
tr_np.columns = col_names

In [76]:
sample_dicts = {}

In [83]:
for column in tr_np:
    thresholds = np.percentile(tr_np[column], [20, 40, 60, 80])
    gene_scores = {}
    for gene, expression in tr_np[column].items():
        if expression > thresholds[3]: 
            score = 3
        elif expression > thresholds[2]: 
            score = 2
        elif expression > thresholds[1]: 
            score = 1
        elif expression > thresholds[0]: 
            score = 0
        else: 
            score = -1
        
        gene_scores[gene] = score
    
    sample_dicts[column] = gene_scores
    
rxns_dicts = {}

for sample, gene_scores in sample_dicts.items():
    rxn_confidences = {}
    for reaction in mod.reactions:
        rule = reaction.gene_reaction_rule
        conf_score = reaction_confidence(rule, gene_scores)
        rxn_confidences[reaction.id] = conf_score
    
    rxns_dicts[sample] = rxn_confidences
    
media_comp = [
    "MAR09034", # Glucose
    "MAR09035", # Linoleate
    "MAR09036", # Linolenate
    "MAR09038", # His
    "MAR09039", # Ile
    "MAR09040", # Leu
    "MAR09041", # Lys
    "MAR09042", # Met
    "MAR09043", # Phe
    "MAR09044", # Thr
    "MAR09045", # Trp
    "MAR09046", # Val
    "MAR09047", # H2O
    "MAR09048", # O2
    "MAR09061", # Ala
    "MAR09062", # Asn
    "MAR09063", # Gln
    "MAR09064", # Tyr
    "MAR09065", # Cys
    "MAR09066", # Arg
    "MAR09067", # Gly
    "MAR09068", # Pro
    "MAR09069", # Ser
    "MAR09070", # Asp
    "MAR09071", # Glu
    "MAR09072", # Phosphate
    "MAR09076", # Fe2+
    "MAR09109", # Biotin
    "MAR09135", # Lactate
    "MAR09143", # Riboflavin
    "MAR09144", # Pyridoxine
    "MAR09145", # Pantothenate
    "MAR09146", # Folate
    "MAR09147", # Retinol
    "MAR09150", # Chloride
    "MAR09151", # a-tocopherole
    "MAR09153", # y-tocopherole
    "MAR09158", # ascorbate
    "MAR09159", # thiamine
    "MAR09167", # lipoic acid
    "MAR09269", # aquacob(III)alamin
    "MAR11420" # Ammonia
    ]

secr_comp = ["MAR09422", # Thymine
             "MAR09142", # Nicotinate
             "MAR09437", # Uracil
             "MAR09378", # Nicotinamide
             "MAR11428", # Xanthine
             "MAR09358", # Hypoxanthine
             "MAR09021", # 5-Methyladenosine
             "MAR09253", # Adenine
             "MAR09341", # Betaine
             "MAR09353", # Guanine
             "MAR09848", # Dimethylglycine
             "MAR09290", # Creatine
             "MAR09131", # Sarcosine
             "MAR10428", # 5-Aminoevulinic acid
             "MAR04849", # Citrulline
             "MAR09087", # Ornithine
             "MAR11400", # Fumarate
             "MAR09415", # Succinate
             "MAR09025", # Pyroglutamate / 5-oxoproline
             "MAR11404", # Malate
             "MAR09286", # Citrate
             "MAR09861", # Xanthosine
             "MAR10431", # Allantoine
             "MAR09285" # Cholesterole
            ]

biomass_rxns = ["MAR13082", # Biomass
                "MAR10023", # Biomass export
                "MAR10024", # Biomass export
                "MAR07160", # DNA Pool reaction
                "MAR13086", # RNA Pool reaction
                "MAR10065", # Cofactor Pool reaction
                "MAR09727", # Glycogen Pool reaction
                "MAR10063", # Lipid Pool reaction
                "MAR10064", # Metabolite Pool reaction
                "MAR10062" # Protein Pool reaction
                ]

for sample in rxns_dicts:
    for reaction_id in media_comp:
        # Set the confidence score to 3 for each specified reaction
        rxns_dicts[sample][reaction_id] = 3

for sample in rxns_dicts:
    for reaction_id in secr_comp:
        # Set the confidence score to 3 for each specified reaction
        rxns_dicts[sample][reaction_id] = 3

for sample in rxns_dicts:
    for reaction_id in biomass_rxns:
        # Set the confidence score to 3 for each specified reaction
        rxns_dicts[sample][reaction_id] = 3

In [ ]:
print("Coonfidence scores successfully assigned")

for r in mod.exchanges:
    mod.reactions.get_by_id(r.id).lower_bound = 0.0

for r in media_comp:
    mod.reactions.get_by_id(r).lower_bound = -1000.0

mod.solver = 'cplex'

reconstructed_models = {}

# Iterate through each sample in rxns_dicts
for sample, conf_scores in rxns_dicts.items():
    # Initialize CORDA with the original model and the sample-specific confidence scores
    corda_model = CORDA(mod, conf_scores)
    
    # Build the model
    corda_model.build()
    
    # Retrieve the reconstructed COBRA model
    model_reconstr = corda_model.cobra_model()
    
    # Store the reconstructed model with the sample name as the key
    reconstructed_models[sample] = model_reconstr
    
    print(f"Reconstruction done for {sample}, with ", len(model_reconstr.reactions), "reactions")

print("Reconstruction of all models successfull")    

reaction_ids = [reaction.id for reaction in mod.reactions]
binary_matrix = pd.DataFrame(index=reaction_ids)

# Iterate through each reconstructed model in 'reconstructed_models'
for sample_name, model_reconstr in reconstructed_models.items():
    # Initialize a column for the current sample with zeros
    binary_matrix[sample_name] = 0
    
    # Update the column with 1s where reactions are present in the reconstructed model
    for reaction in model_reconstr.reactions:
        if reaction.id in binary_matrix.index:
            binary_matrix.loc[reaction.id, sample_name] = 1

# Define the output directory and ensure it exists
output_dir = "/home/users/lzehetner/data/paper4_aav/HEK_Dietmair"
os.makedirs(output_dir, exist_ok=True)

# Define the full output path and save the DataFrame as a CSV file
output_path = os.path.join(output_dir, "reaction_matrix.csv")
binary_matrix.to_csv(output_path)

print(f"Binary matrix saved to {output_path}")